In [10]:
!pip install selenium
!apt-get update
!apt-get install firefox -y
!pip install selenium
# Download and install GeckoDriver
!wget https://github.com/mozilla/geckodriver/releases/download/v0.30.0/geckodriver-v0.30.0-linux64.tar.gz
!tar -xvzf geckodriver-v0.30.0-linux64.tar.gz
!chmod +x geckodriver
!mv geckodriver /usr/local/bin/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 3s (127 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

In [11]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from google.colab import drive
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gc
import time
import pytz
import datetime
import logging
from tqdm import tqdm
import traceback

In [12]:
gdown.download(id = '1DuT3nnIouLKuRFbGo5lqcQwKTTb5GM8r')
df = pd.read_csv('/content/suburbs.csv')

Downloading...
From: https://drive.google.com/uc?id=1DuT3nnIouLKuRFbGo5lqcQwKTTb5GM8r
To: /content/drive/MyDrive/Files-Required/client project/suburbs.csv

100%|██████████| 975k/975k [00:00<00:00, 43.1MB/s]


In [13]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Files-Required/client project')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
df.shape

(15879, 2)

In [ ]:
def driver_setup():
    options = webdriver.FirefoxOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Firefox(options=options)
    return driver

log_filename = 'exemption_logs.txt'
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
file_handler = logging.FileHandler(log_filename)
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logging.getLogger().addHandler(file_handler)

def get_data_from_row(row, soup):
    # Extract data from a row in the table
    cells = row.find_all("td")
    bedrooms = cells[0].text.strip()
    prop_type = cells[1].text.strip()
    median_price = cells[2].text.strip()
    days_on_market = cells[3].text.strip()
    clearance_rate = cells[4].text.strip()
    sold_this_year = cells[5].text.strip()
    population = soup.find('div', {'class': 'css-48zwbo'}).find('div', {'class': 'css-54bw0x'}).text
    average_age = soup.find_all('div', {'class': 'css-48zwbo'})[1].find('div', {'class': 'css-54bw0x'}).text
    owner_percent = soup.find('span', {'data-testid': 'left-value'}).text.strip()
    renter_percent = soup.find('span', {'data-testid': 'right-value'}).text.strip()
    family_percent = soup.find_all('span', {'data-testid': 'left-value'})[1].text.strip()
    single_percent = soup.find_all('span', {'data-testid': 'right-value'})[1].text.strip()
    australia_timezone = pytz.timezone('Australia/Sydney')
    current_time = datetime.datetime.now(australia_timezone)
    return {
        "Bedrooms": bedrooms,
        "Type": prop_type,
        "Median Price": median_price,
        "Avg Days on Market": days_on_market,
        "Clearance Rate": clearance_rate,
        "Sold This Year": sold_this_year,
        'POPULATION': population,
        'AVERAGE AGE': average_age,
        'OWNER': owner_percent,
        'RENTER': renter_percent,
        'FAMILY': family_percent,
        'SINGLE': single_percent,
        'SUBURB': df['0'][i],
        'Time_stamp': current_time
    }


def click_button_and_get_title(driver, row_index, row):
    try:
        buttons = driver.find_elements(By.XPATH, "//button[@title='Open']")
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[@title='Open']")))
        time.sleep(2)
        buttons[row_index].click()
        WebDriverWait(driver, 10).until(EC.staleness_of(buttons[row_index]))
        time.sleep(2)
        button_html = buttons[row_index+1].get_attribute('outerHTML')
        soup = BeautifulSoup(button_html, 'html.parser')
        title_element = soup.find('div').find('title')
        title = title_element.get_text()

        # Retrieve the entire page source
        html = driver.page_source
        subsoup = BeautifulSoup(html, 'html.parser')
        tr_elements = subsoup.find_all('tr')
        datad = {}

        for tr_element in tr_elements:
            div_elements = tr_element.find_all('div')  # Find all <div> elements within the <tr> element
            for div_element in div_elements:
                text_content = div_element.get_text(strip=True)
                text_content = text_content.lower()

                if 'Sold this year'.lower() in text_content:
                    sold_this_year = div_element.find_next('div').get_text(strip=True)
                    datad['Sold this year'] = sold_this_year
                elif 'Average days on market'.lower() in text_content:
                    avg_days_on_market = div_element.find_next('div').get_text(strip=True)
                    datad['Average days on market'] = avg_days_on_market
                elif 'Auction clearance'.lower() in text_content:
                    auction_clearance = div_element.find_next('div').get_text(strip=True)
                    datad['Auction clearance'] = auction_clearance
                elif 'Rental median price'.lower() in text_content:
                    rental_median_price = div_element.find_next('div').get_text(strip=True)
                    datad['Rental median price'] = rental_median_price

        cleaned_data = {}
        for key, value in datad.items():
            cleaned_key = key.replace(':\xa0', '')
            cleaned_data[cleaned_key] = value

    except Exception as e:
        traceback.print_exc()
        return {}, None

    tabled = subsoup.find('table', {'id': 'suburb-insights__table'})
    return cleaned_data, tabled


def table_and_vec(table):
    try:
        rows = table.find_all('tr')
        data = {}
        for row in rows[1:]:  # Skip the header row
            columns = row.find_all('td')
            year = columns[0].text.strip()
            median = columns[1].text.strip()
            growth = columns[2].text.strip()
            sales = columns[3].text.strip()
            data[f'median price {year}'] = median
            data[f'growth rate {year}'] = growth
            data[f'no of sales {year}'] = sales
    except Exception as e:
        logging.error(str(e))
        return {}

    return data

def save_data_to_disk(data, filename):
    if os.path.isfile(filename):
        existing_df = pd.read_csv(filename)
        data = pd.concat([existing_df, data], ignore_index=True)
        os.remove(filename)
        data.to_csv(filename,index=False)
    else:
        data.to_csv(filename, mode='w', header=True, index=False)

st_rng = 8464
end_rng = 9522
total_rng = abs(end_rng - st_rng)

pbar = tqdm(total=total_rng, desc='Searching', unit='query')

for i in range(st_rng , end_rng):
    pbar.update(1)
    logging.info(f"Processing index {i}")
    try:
        url = 'https://www.domain.com.au/suburb-profile/' + df['0'][i]
        driver = driver_setup()
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find("table")
        rows = table.find_all("tr")[1:]

        for row_index, row in enumerate(rows):

          rodict = {}

          row_data = None
          table = None
          buton_records = None
          suburbtable = None
          try:
            row_data = get_data_from_row(row, soup)
          except:
            None
          if row_data != None:
            try:
              buton_records, table = click_button_and_get_title(driver, row_index, row)
            except:
              None
          if table != None:
            try:
              suburbtable = table_and_vec(table)
            except:
              None

          driver.delete_all_cookies()

          if row_data != None:
            rodict.update(row_data)
          if buton_records != 0:
            rodict.update(buton_records)
          if suburbtable != 0:
            rodict.update(suburbtable)

          if rodict != {}:
            australia_timezone = pytz.timezone('Australia/Sydney')
            current_time = datetime.datetime.now(australia_timezone)
            tempdf = pd.DataFrame(rodict, index=[0])
            tempfilename = 'Temp_Domain_Extract_'+ str(current_time.date()).replace('-', '')+'.csv'
            save_data_to_disk(tempdf, tempfilename)
            tempdf = pd.DataFrame()
            driver.delete_all_cookies()

    except Exception as e:
        logging.error(str(e))

    driver.quit()
    gc.collect()
    os.system("taskkill /f /im firefox.exe")


Searching:   0%|          | 4/1058 [16:06<70:46:14, 241.72s/query]
ERROR:root:'NoneType' object is not iterable

Searching:   0%|          | 2/1058 [00:15<2:18:35,  7.87s/query]ERROR:root:Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:395:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:134:16


Searching:   0%|          | 3/1058 [00:38<4:10:10, 14.23s/query]Traceback (most recent call last):
  File "<ipython-input-15-cfcbea41692b>", line 58, in click_button_and_get_title
    WebDriverWait(driver, 10).until(EC.staleness_of(buttons[row_index]))
  File "/usr/local/lib/python3.10/dist-packages/selenium/webdriver/support/wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 

ERROR:root:'NoneTyp